In [24]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch

from modechoice.config import MODES
from modechoice.io import load_bundle
from modechoice.pipeline import ModeChoicePipeline
from modechoice.model import NestedLogitHetero
from modechoice.tensors import Standardizer
from modechoice.data import ensure_long_format
from modechoice.features import add_relative_features_long
from modechoice.tensors import build_choice_tensors_hetero

from modechoice.pricing_user import make_user_tensor_hetero, user_probs, optimize_user

In [ ]:
ROOT = Path.cwd()
if not (ROOT / "pyproject.toml").exists():
    ROOT = ROOT.parent

DEVICE = "cpu"
torch.set_default_dtype(torch.float32)

BUNDLE_DIR = ROOT / "artifacts" / "modechoice_bundle"
DATA_PATH  = ROOT / "dataset" / "ModeCanada.csv"

# print("ROOT:", ROOT)
print("BUNDLE_DIR exists:", BUNDLE_DIR.exists())
print("DATA_PATH exists:", DATA_PATH.exists())

BUNDLE_DIR exists: True
DATA_PATH exists: True


In [26]:
pipe = load_bundle(
    BUNDLE_DIR,
    PipelineCls=ModeChoicePipeline,
    ModelCls=NestedLogitHetero,
    ScalerCls=Standardizer,
)

pipe.model.asc, pipe.model.beta  # sanity
print("Loaded feat_names:", len(pipe.scaler.feat_names))
print("First 10 feats:", pipe.scaler.feat_names[:10])
print("Lambdas:", pipe.model.lambdas())

Loaded feat_names: 14
First 10 feats: ['cost', 'ivt', 'ovt', 'freq', 'urban_x_ovt', 'gen_time', 'income_train', 'urban_train', 'income_car', 'urban_car']
Lambdas: {'air': 0.6149392127990723, 'land': 0.9137387871742249}


In [ ]:
pipe.scaler.mu = np.asarray(pipe.scaler.mu, dtype=np.float32)
pipe.scaler.sd = np.asarray(pipe.scaler.sd, dtype=np.float32)

feat_names = pipe.scaler.feat_names

per_mode_feats = set()
for m in MODES:
    per_mode_feats.add(f"income_{m}")
    per_mode_feats.add(f"urban_{m}")

item_feat_names = [f for f in feat_names if f not in per_mode_feats]

print("Derived item_feat_names:", item_feat_names)
print("Derived D_item:", len(item_feat_names))
print("Total D_model:", len(feat_names))

Derived item_feat_names: ['cost', 'ivt', 'ovt', 'freq', 'urban_x_ovt', 'gen_time']
Derived D_item: 6
Total D_model: 14


In [ ]:
df_raw = pd.read_csv(DATA_PATH)
df_long = ensure_long_format(df_raw)

df_long = add_relative_features_long(df_long, w_ovt=2.0, freq_period_minutes=1440.0)

all_t, _ = build_choice_tensors_hetero(df_long, scaler=pipe.scaler, fit_scaler=False, item_feat_names=item_feat_names)

assert all_t["feat_names"] == pipe.scaler.feat_names, "Feature mismatch vs trained scaler!"

print("all_t keys:", all_t.keys())
print("X_item:", all_t["X_item"].shape)
print("X_item_orig:", all_t["X_item_orig"].shape)
print("avail:", all_t["avail"].shape)
print("y:", all_t["y"].shape)

all_t keys: dict_keys(['X_item', 'X_item_orig', 'avail', 'y', 'cases', 'feat_names'])
X_item: torch.Size([4324, 4, 14])
X_item_orig: (4324, 4, 14)
avail: torch.Size([4324, 4])
y: torch.Size([4324])


In [ ]:
# include ALL item_feat_names used by the trained model
# set to 0.0 if no values (but better compute properly)
base_item = {
    "train": {"cost": 55, "ivt": 60, "ovt": 10, "freq": 4, "urban_x_ovt": 10, "gen_time": 80},
    "car":   {"cost": 40, "ivt": 75, "ovt":  0, "freq": 0, "urban_x_ovt": 0,  "gen_time": 75},
    "bus":   {"cost": 25, "ivt": 95, "ovt": 15, "freq": 8, "urban_x_ovt": 15, "gen_time": 110},
    "air":   {"cost": 160,"ivt": 50, "ovt": 40, "freq": 3, "urban_x_ovt": 40, "gen_time": 90},
}

case_features = {"income": 70, "urban": 1}

Xstd_u, avail_u, Xorig_u = make_user_tensor_hetero(
    base_item=base_item,
    item_feat_names=item_feat_names,
    case_features=case_features,
    scaler=pipe.scaler,
)

asc, beta = pipe.model.asc, pipe.model.beta
raw_la, raw_ll = pipe.model.raw_lam_air, pipe.model.raw_lam_land

p0 = user_probs(Xstd_u, avail_u, asc, beta, raw_la, raw_ll)
print("Baseline probs:", {m: float(p0[i]) for i, m in enumerate(MODES)})

Baseline probs: {'train': 0.5207672119140625, 'car': 0.4381403625011444, 'bus': 0.002354599768295884, 'air': 0.038737814873456955}


In [30]:
mult_all, base_cost, new_cost, probs = optimize_user(
    X_user_orig_t=Xorig_u,
    avail_user_t=avail_u,
    scaler=pipe.scaler,
    asc=asc, beta=beta, raw_la=raw_la, raw_ll=raw_ll,
    control_modes=("train","air"),
    target_modes=("train","air"),
    objective="prob",
    mult_bounds=(0.8, 1.2),
    steps=250,
    lr=0.08,
    forbid_modes=("car",),
)

out = pd.DataFrame({
    "mode": MODES,
    "multiplier": mult_all,
    "base_cost": base_cost,
    "new_cost": new_cost,
    "prob": probs,
})
out["expected_revenue"] = out["new_cost"] * out["prob"]
out

,mode,multiplier,base_cost,new_cost,prob,expected_revenue
0,train,0.802092,55.0,44.115059,0.592618,26.143393
1,car,1.000000,40.0,40.000000,0.300161,12.006446
2,bus,1.000000,25.0,25.000000,0.001613,0.040327
3,air,0.801241,160.0,128.198578,0.105607,13.538717


In [33]:
mult_all, base_cost, new_cost, probs = optimize_user(
    X_user_orig_t=Xorig_u,
    avail_user_t=avail_u,
    scaler=pipe.scaler,
    asc=asc, beta=beta, raw_la=raw_la, raw_ll=raw_ll,
    control_modes=("train","air"),
    target_modes=("train","air"),   # doesn't matter for revenue objective
    objective="revenue",
    mult_bounds=(0.8, 1.2),
    steps=250,
    lr=0.08,
    forbid_modes=("car",),
)

out = pd.DataFrame({
    "mode": MODES,
    "multiplier": mult_all,
    "base_cost": base_cost,
    "new_cost": new_cost,
    "prob": probs,
})
out["expected_revenue"] = out["new_cost"] * out["prob"]
display(out)
float(out["expected_revenue"].sum())

,mode,multiplier,base_cost,new_cost,prob,expected_revenue
0,train,1.162964,55.0,63.963001,0.368292,23.557045
1,car,1.000000,40.0,40.000000,0.470585,18.823404
2,bus,1.000000,25.0,25.000000,0.002529,0.063224
3,air,0.801092,160.0,128.174652,0.158594,20.327761


62.77143096923828